In [1]:
struct Tracker
    val
end

function Untrack(X::⬅Tracker...)
    return [x.val for x in X]
end



UndefVarError: UndefVarError: `⬅Tracker` not defined

In [2]:
macro Dualcall(args...)
    args_expr = Expr(:vect, args...) # Construct a vector of arguments

    # Construct the loop body using correct interpolation and iteration
    body = quote
        Z, Chainer = ⬅Dual($args_expr...)
        Z = ⬅Tracker(Z)
    end

    return body
end

println(macroexpand(Main, :(@Dualcall(X, Y))))



begin
    #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:6 =#
    (var"#64#Z", var"#63#Chainer") = Main.⬅Dual([Main.X, Main.Y]...)
    #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:7 =#
    var"#64#Z" = Main.⬅Tracker(var"#64#Z")
end


In [3]:

macro Chaincalls(args...)
    args_expr = Expr(:vect, args...) # Construct a vector of arguments

    # Construct the loop body using correct interpolation and iteration
    loop_body = quote
        for (i, arg) in enumerate($args_expr)
            push!(arg.Chainers, ∇ -> Chainer(∇)[i])
            push!(arg.Childs, Z.id)
            push!(Tape, arg)
        end
    end

    return loop_body
end


println(macroexpand(Main, :(@Chaincalls(X, Y))))

begin
    #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:7 =#
    for (var"#65#i", var"#66#arg") = Main.enumerate([Main.X, Main.Y])
        #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:8 =#
        Main.push!((var"#66#arg").Chainers, ((var"#67#∇",)->begin
                    #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:8 =#
                    (Main.Chainer(var"#67#∇"))[var"#65#i"]
                end))
        #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:9 =#
        Main.push!((var"#66#arg").Childs, (Main.Z).id)
        #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCl

In [4]:
macro Overloading(func, args, dualcall, chaincalls)
    # Prepare the arguments tuple for function definition
    args_def = eval(args)  # This evaluates the args tuple to use in the function signature

    # Prepare the dualcall and chaincalls
    # Note: Direct embedding of macro calls in generated code can be tricky due to expansion times
    # It might be more reliable to directly insert the intended logic or call functions
    
    # Generate the function definition
    func_def = quote
        function $(esc(func))($(esc(args_def))...)
            # Assuming dualcall and chaincalls are function calls or contain executable logic
            $(esc(dualcall))
            $(esc(chaincalls))
        end
    end
    
    return func_def
end

@Overloading f(x::Tracker, y::Tracker) @Dualcall(x, y) @Chaincalls(x, y)


LoadError: LoadError: MethodError: no method matching var"@Overloading"(::LineNumberNode, ::Module, ::Expr, ::Expr, ::Expr)

Closest candidates are:
  var"@Overloading"(::LineNumberNode, ::Module, ::Any, ::Any, ::Any, !Matched::Any)
   @ Main ~/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:1

in expression starting at /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:21

In [5]:
Dualcall = quote
    Z, Chainer = ⬅Dual(X, Y)
    Z = ⬅Tracker(Z)
end



quote
    #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:2 =#
    (Z, Chainer) = ⬅Dual(X, Y)
    #= /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:3 =#
    Z = ⬅Tracker(Z)
end

In [6]:
macro ⬅Overload(fname, args, dualcall=nothing, chaincalls=nothing)
    # Prepare arguments for the ⬅Dual function and chaining
    untracked_args = Expr(:vect, [:(Untrack($arg)) for arg in args.args]...)
    tracked_args = Expr(:vect, [arg for arg in args.args if occursin("Tracker", string(arg))])

    # Construct the dual call if not provided
    dual_call_expr = dualcall === nothing ? :(⬅Dual($fname, $untracked_args...)) : dualcall

    # Construct the chaining logic if not provided
    chain_calls_expr = chaincalls === nothing ? quote
        for arg in $tracked_args
            push!(arg.Chainers, ∇ -> Chainer(∇)[i])
            push!(arg.Childs, Z.id)
        end
    end : chaincalls

    # Construct the function body
    func_body = quote
        Z, Chainer = $dual_call_expr
        Z = ⬅Tracker(Z)
        $chain_calls_expr
        return Z
    end

    return esc(:(function $fname($args) $func_body end))
end
⬅Dual(g, X, Y) = (0, _ -> (0, 0))


@⬅Overload g(X::Tracker, Y::Tracker) nothing nothing

g(Tracker(0), Tracker(0))

LoadError: LoadError: type Symbol has no field args
in expression starting at /Users/pichedereck/Library/Mobile Documents/iCloud~md~obsidian/Documents/MemorisiCloud/🟣 Personnel/📟 Code/Utero/test/MacroOperations.ipynb:30